# Similarity Evaluation Analysis (SEA) Dataset D

In [1]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
print('Libraries imported!!')

Libraries imported!!


In [2]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/PRIVACY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for univariate resemblance analisys
from similarity_evaluation import scale_data
from similarity_evaluation import pairwise_euclidean_distance
from similarity_evaluation import hausdorff_distance
from similarity_evaluation import rts_similarity

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read real and synthetic datasets
In this part real and synthetic datasets are read.

In [3]:
#Define global variables
DATA_TYPES = ['Real','GM','SDV','CTGAN','WGANGP']
SYNTHESIZERS = ['GM','SDV','CTGAN','WGANGP']
FILEPATHS = {'Real' : HOME_PATH + 'REAL DATASETS/TRAIN DATASETS/D_ContraceptiveMethod_Real_Train.csv',
            'GM' : HOME_PATH + 'SYNTHETIC DATASETS/GM/D_ContraceptiveMethod_Synthetic_GM.csv',
            'SDV' : HOME_PATH + 'SYNTHETIC DATASETS/SDV/D_ContraceptiveMethod_Synthetic_SDV.csv',
            'CTGAN' : HOME_PATH + 'SYNTHETIC DATASETS/CTGAN/D_ContraceptiveMethod_Synthetic_CTGAN.csv',
            'WGANGP' : HOME_PATH + 'SYNTHETIC DATASETS/WGANGP/D_ContraceptiveMethod_Synthetic_WGANGP.csv'}
categorical_columns = ['wife_education','husband_education','wife_religion','wife_working','husband_occupation',
                        'standard_of_living_index','media_exposure','contraceptive_method_used']
data = dict()

In [4]:
#iterate over all datasets filepaths and read each dataset
for name, path in FILEPATHS.items() :
    data[name] = pd.read_csv(path)
    for col in categorical_columns :
        data[name][col] = data[name][col].astype('category').cat.codes
data

{'Real':       wife_age  wife_education  husband_education  children  wife_religion  \
 0           36               1                  3        11              1   
 1           48               0                  2         4              1   
 2           38               0                  1         8              1   
 3           27               1                  1         5              1   
 4           26               2                  1         4              1   
 ...        ...             ...                ...       ...            ...   
 1173        34               1                  3         6              1   
 1174        23               3                  3         1              1   
 1175        29               2                  3         3              1   
 1176        37               3                  3         5              0   
 1177        31               3                  3         2              1   
 
       wife_working  husband_occupation  s

## 2. Normalize data

In [5]:
#Scale the data
num_cols = (data['Real'].select_dtypes(include=['int64','float64'])).columns
scaled_data = dict()
for name in DATA_TYPES :
    scaled_data[name] = scale_data(data[name])
scaled_data

{'Real':       wife_age  wife_education  husband_education  children  wife_religion  \
 0     0.606061        0.333333           1.000000    0.6875            1.0   
 1     0.969697        0.000000           0.666667    0.2500            1.0   
 2     0.666667        0.000000           0.333333    0.5000            1.0   
 3     0.333333        0.333333           0.333333    0.3125            1.0   
 4     0.303030        0.666667           0.333333    0.2500            1.0   
 ...        ...             ...                ...       ...            ...   
 1173  0.545455        0.333333           1.000000    0.3750            1.0   
 1174  0.212121        1.000000           1.000000    0.0625            1.0   
 1175  0.393939        0.666667           1.000000    0.1875            1.0   
 1176  0.636364        1.000000           1.000000    0.3125            0.0   
 1177  0.454545        1.000000           1.000000    0.1250            1.0   
 
       wife_working  husband_occupation  s

## 3. Calculate the Euclidean distances between each pair of values

In [6]:
distances_values = dict()

for name in SYNTHESIZERS :
    #distances = distance.cdist(scaled_data[name].values, real_data_scaled, 'euclidean')
    distances_values[name] = pairwise_euclidean_distance(scaled_data[name].values, scaled_data['Real'].values)
    
distances_values

{'GM': '1.384 ± 0.3853',
 'SDV': '1.3493 ± 0.3699',
 'CTGAN': '1.4468 ± 0.3693',
 'WGANGP': '1.6469 ± 0.3333'}

## 4. Calculate the Hausdorff distance between synthetic data and real data

In [7]:
hausdorff_values = dict()

for name in SYNTHESIZERS :
    hausdorff_values[name] = hausdorff_distance(scaled_data[name].values, scaled_data['Real'].values)
    
hausdorff_values

{'GM': 1.0232, 'SDV': 0.8778, 'CTGAN': 1.1205, 'WGANGP': 0.8922}

## 5. Calculate maximum RTS similarity

In [8]:
str_values = dict()

for name in SYNTHESIZERS :
    str_values[name] = rts_similarity(scaled_data[name].values, scaled_data['Real'].values)
    
str_values

{'GM': {'min': 0.0404, 'mean': 0.7754, 'max': 1.0},
 'SDV': {'min': 0.0273, 'mean': 0.7986, 'max': 0.9999},
 'CTGAN': {'min': 0.0051, 'mean': 0.7493, 'max': 1.0},
 'WGANGP': {'min': 0.0324, 'mean': 0.652, 'max': 0.9999}}